In [22]:
import os
import glob
import csv
import numpy as np
import trimesh
import matplotlib.pyplot as plt

In [23]:
INPUT_DIR = "meshes"
OUTPUT_DIR = "outputs"
PLOT_DIR = "plots"
SUMMARY_CSV = os.path.join(OUTPUT_DIR,"summary.csv")
BINS = 1024

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

In [24]:
print("Output directory absolute path:", os.path.abspath(OUTPUT_DIR))
print("PLOTS directory absolute path:", os.path.abspath(PLOT_DIR))

Output directory absolute path: c:\Users\user\Desktop\mesh_Assignment\outputs
PLOTS directory absolute path: c:\Users\user\Desktop\mesh_Assignment\plots


In [25]:
def min_max_norm(V):
    vmin = V.min(axis=0)
    vmax = V.max(axis=0)
    span = np.where((vmax-vmin) == 0, 1.0, (vmax-vmin))
    print("Span was called for V: ")
    print(span)
    Vn = (V-vmin)/span
    ctx = {"type":"minmax", "vmin":vmin, "vmax":vmax}
    return Vn,ctx

def min_max_denorm(Vn,ctx):
    vmin,vmax = ctx["vmin"],ctx["vmax"]
    span = np.where((vmax-vmin)==0, 1.0 , (vmax-vmin))
    return Vn*span + vmin

### min_max_norm and min_max_denorm — Explanation

Purpose
- Provide a simple column-wise min–max normalization and its inverse (denormalization) for NumPy arrays.

min_max_norm(V)
- Input: V — a NumPy array (typically shape (n, d)). Normalization is applied column-wise (axis=0).
- What it does:
    - Computes column-wise minimum `vmin = V.min(axis=0)` and maximum `vmax = V.max(axis=0)`.
    - Computes `span = vmax - vmin`. To avoid division by zero for constant columns, any zero spans are replaced with `1.0`.
    - Prints the computed `span` (side effect).
    - Produces normalized values `Vn = (V - vmin) / span`, so each column is mapped into [0, 1] when span > 0.
    - Returns a tuple `(Vn, ctx)` where `ctx` is a dictionary containing:
        - `"type": "minmax"`
        - `"vmin": vmin`
        - `"vmax": vmax`
- Notes and edge cases:
    - For a constant column (vmax == vmin) the code uses `span = 1.0`, so `Vn` will be all zeros for that column (since V - vmin == 0). This avoids NaNs/Infs.
    - The function prints the span array; remove the print if silent operation is desired.

min_max_denorm(Vn, ctx)
- Input:
    - Vn — normalized NumPy array (same shape as original).
    - ctx — the context dictionary returned by `min_max_norm`.
- What it does:
    - Recomputes `span = vmax - vmin` with the same zero-handling (uses 1.0 when span == 0).
    - Returns denormalized values `V = Vn * span + vmin`, recovering the original scale.
- Requirements:
    - `ctx` must contain `"vmin"` and `"vmax"` as produced by `min_max_norm`.
    - Works column-wise; shape of `Vn` should be compatible with `vmin`/`vmax`.

Small example
```
# V = [[0, 10],
#      [5, 20]]
# vmin = [0, 10], vmax = [5, 20], span = [5, 10]
# Vn = [[0, 0], [1, 1]]
# denorm(Vn, ctx) -> original V
```

Behavior summary
- Use `min_max_norm` to scale data to 0–1 per column and keep the context for exact inverse scaling.
- `min_max_denorm` recovers original values using that context, safely handling constant columns.

In [26]:
def unitsphere_norm(V):
    centroid = V.mean(axis=0)
    Vc = V - centroid
    scale = np.max(np.linalg.norm(Vc,axis = 1))
    scale = 1.0 if scale == 0 else scale
    Vn = Vc / scale
    Vn01 = (Vn+1.0)/2.0
    ctx = {"type":"unitsphere", "centroid":centroid, "scale":scale}

    return Vn01,ctx

def unitsphere_denorm(Vn01,ctx):
    Vn = Vn01*2.0 - 1.0
    return Vn * ctx["scale"] + ctx["centroid"]

### unitsphere_norm and unitsphere_denorm — Explanation

Purpose
- Normalize 3D points to fit within a unit sphere, then map to [0,1]³ range.
- Provide inverse transformation to recover original coordinates.

unitsphere_norm(V)
1. Center the points:
    - $\mathbf{c} = \frac{1}{n}\sum_{i=1}^n \mathbf{v}_i$ (centroid)
    - $\mathbf{V}_c = \mathbf{V} - \mathbf{c}$ (centered points)

2. Scale to unit sphere:
    - $s = \max_i \|\mathbf{v}_{c,i}\|_2$ (maximum distance from centroid)
    - If $s = 0$, set $s = 1$ to handle degenerate case
    - $\mathbf{V}_n = \mathbf{V}_c / s$ (normalized to unit sphere)

3. Map to [0,1]³:
    - $\mathbf{V}_{01} = (\mathbf{V}_n + 1) / 2$

unitsphere_denorm(Vn01, ctx)
1. Map back to [-1,1]³:
    - $\mathbf{V}_n = 2\mathbf{V}_{01} - 1$

2. Restore original scale and position:
    - $\mathbf{V} = s\mathbf{V}_n + \mathbf{c}$

Properties
- Preserves relative distances and angles after centering
- Guarantees all points fit within a unit cube after normalization
- Perfectly reversible transformation

In [27]:
# def quantize(Vn01, bins=BINS):
#     # First ensure values are between 0 and 1
#     clipped = np.zeros_like(Vn01)
#     for i in range(len(Vn01)):
#         for j in range(len(Vn01[i])):
#             if Vn01[i,j] < 0.0:
#                 clipped[i,j] = 0.0
#             elif Vn01[i,j] > 1.0:
#                 clipped[i,j] = 1.0
#             else:
#                 clipped[i,j] = Vn01[i,j]
    
#     # Scale to bins-1 and round down to integers
#     quantized = np.zeros_like(clipped)
#     for i in range(len(clipped)):
#         for j in range(len(clipped[i])):
#             # Scale to [0, bins-1] range
#             scaled = clipped[i,j] * (bins - 1)
#             # Round down to nearest integer
#             quantized[i,j] = int(scaled)
            
#     return quantized.astype(np.int32)

# def dequantize(Q, bins=BINS):
#     # Convert back to float values in [0,1] range
#     dequantized = np.zeros_like(Q, dtype=np.float32)
#     for i in range(len(Q)):
#         for j in range(len(Q[i])):
#             dequantized[i,j] = float(Q[i,j]) / (bins - 1)
            
#     return dequantized

def quantize(Vn01, bins=BINS):
    Q = np.floor(np.clip(Vn01, 0.0, 1.0) * (bins - 1)).astype(np.int32)
    return Q

def dequantize(Q, bins=BINS):
    return Q.astype(np.float32) / (bins - 1)

In [28]:
def mse(A, B):
    return np.mean((A - B) ** 2, axis=0), np.mean((A - B) ** 2)

def mae(A, B):
    return np.mean(np.abs(A - B), axis=0), np.mean(np.abs(A - B))

### Mathematical definitions — MSE and MAE

Let A,B ∈ R^{n×d} with entries a_{i,j}, b_{i,j} for i=1..n, j=1..d. Define the per-entry error e_{i,j} = a_{i,j} − b_{i,j}.

- Per-dimension Mean Squared Error (MSE)  
    MSE_j = (1/n) ∑_{i=1}^n (a_{i,j} − b_{i,j})^2, for j = 1..d

- Overall MSE (averaged over all entries)  
    MSE_total = (1/(n d)) ∑_{j=1}^d ∑_{i=1}^n (a_{i,j} − b_{i,j})^2  
    (equivalently the mean of all entries of (A−B)²)

- Per-dimension Mean Absolute Error (MAE)  
    MAE_j = (1/n) ∑_{i=1}^n |a_{i,j} − b_{i,j}|, for j = 1..d

- Overall MAE (averaged over all entries)  
    MAE_total = (1/(n d)) ∑_{j=1}^d ∑_{i=1}^n |a_{i,j} − b_{i,j}|

Notes mapping to the notebook functions:
- `mse(A, B)` returns (vector_of_MSE_per_dimension, scalar_MSE_total).
- `mae(A, B)` returns (vector_of_MAE_per_dimension, scalar_MAE_total).

Optional: Root Mean Squared Error (RMSE) = sqrt(MSE) (applies per-dimension or overall).


In [29]:
def save_mesh(path, vertices, faces):
    m = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
    m.export(path)

In [30]:
def plot_errors(per_axis_values, title, outfile, labels=("X","Y","Z")):
    plt.figure()
    plt.bar(labels, per_axis_values)
    plt.title(title)
    plt.ylabel("Error")
    plt.tight_layout()
    plt.savefig(outfile)
    plt.close()

In [31]:
def stats_str(V):
    return (
        f"count={len(V)} | "
        f"min={V.min(axis=0)} | "
        f"max={V.max(axis=0)} | "
        f"mean={V.mean(axis=0)} | "
        f"std={V.std(axis=0)}"
    )

In [32]:
def process_one(obj_path):
    name = os.path.splitext(os.path.basename(obj_path))[0]
    mesh = trimesh.load(obj_path, process=False)
    V = np.asarray(mesh.vertices, dtype=np.float32)
    F = np.asarray(mesh.faces) if mesh.faces is not None else None

    print(f"\n=== {name} ===")
    print("Original:", stats_str(V))

    results = []

    # ---- Min–Max pipeline ----
    Vn_mm, ctx_mm = min_max_norm(V)
    Q_mm = quantize(Vn_mm)
    Vn_rec_mm = dequantize(Q_mm)
    Vrec_mm = min_max_denorm(Vn_rec_mm, ctx_mm)

    # Save artifacts
    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_normalized_minmax.obj"), Vn_mm, F)
    # “Quantized mesh”: save dequantized vertices (viewer-friendly)
    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_quantized_minmax.obj"), Vn_rec_mm, F)
    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_reconstructed_minmax.obj"), Vrec_mm, F)

    mae_ax, mae_all = mae(V, Vrec_mm)
    mse_ax, mse_all = mse(V, Vrec_mm)
    print(f"Min–Max  | MAE per axis {mae_ax} | MAE {mae_all:.6f} | MSE {mse_all:.6f}")

    plot_errors(mae_ax, f"{name} — MAE per axis (Min–Max)",
                os.path.join(PLOT_DIR, f"{name}_mae_minmax.png"))
    plot_errors(mse_ax, f"{name} — MSE per axis (Min–Max)",
                os.path.join(PLOT_DIR, f"{name}_mse_minmax.png"))

    results.append([name, "minmax", mae_ax[0], mae_ax[1], mae_ax[2], mae_all, mse_ax[0], mse_ax[1], mse_ax[2], mse_all])

    # ---- Unit-Sphere pipeline ----
    Vn_us, ctx_us = unitsphere_norm(V)
    Q_us = quantize(Vn_us)
    Vn_rec_us = dequantize(Q_us)
    Vrec_us = unitsphere_denorm(Vn_rec_us, ctx_us)

    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_normalized_unitsphere.obj"), Vn_us, F)
    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_quantized_unitsphere.obj"), Vn_rec_us, F)
    save_mesh(os.path.join(OUTPUT_DIR, f"{name}_reconstructed_unitsphere.obj"), Vrec_us, F)

    mae_ax, mae_all = mae(V, Vrec_us)
    mse_ax, mse_all = mse(V, Vrec_us)
    print(f"UnitSphere| MAE per axis {mae_ax} | MAE {mae_all:.6f} | MSE {mse_all:.6f}")

    plot_errors(mae_ax, f"{name} — MAE per axis (Unit-Sphere)",
                os.path.join(PLOT_DIR, f"{name}_mae_unitsphere.png"))
    plot_errors(mse_ax, f"{name} — MSE per axis (Unit-Sphere)",
                os.path.join(PLOT_DIR, f"{name}_mse_unitsphere.png"))

    results.append([name, "unitsphere", mae_ax[0], mae_ax[1], mae_ax[2], mae_all, mse_ax[0], mse_ax[1], mse_ax[2], mse_all])

    return results

In [33]:
def main():
    # objs = sorted(glob.glob(os.path.exists(INPUT_DIR), "*.obj"))
    objs = sorted(glob.glob(os.path.join(INPUT_DIR, "*.obj")))
    if not objs:
        print(f"No .obj files found in {INPUT_DIR}/")
        return
    
    # CSV HEADER for WRITE 
    wheader = not os.path.exists(SUMMARY_CSV)
    with open(SUMMARY_CSV, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["mesh","method","mae_x","mae_y","mae_z","mae_all","mse_x","mse_y","mse_z","mse_all"])

        for p in objs:
            print("Processing file:", p)
            rows = process_one(p)
            for r in rows:
                w.writerow(r)
    
    print(f"\nAll done. Outputs in '{OUTPUT_DIR}/', plots in '{PLOT_DIR}/'.")
    print(f"Summary CSV: {SUMMARY_CSV}")

In [34]:
if __name__ == "__main__":
    main()

Processing file: meshes\branch.obj

=== branch ===
Original: count=2767 | min=[-0.851562  0.       -0.464844] | max=[0.849609 1.900391 0.462891] | mean=[0.07544272 1.0873902  0.12196691] | std=[0.34338036 0.45699114 0.20006673]
Span was called for V: 
[1.701171 1.900391 0.927735]
Min–Max  | MAE per axis [0.00081868 0.00093305 0.0004503 ] | MAE 0.000734 | MSE 0.000001
UnitSphere| MAE per axis [0.00129712 0.0013688  0.00130858] | MAE 0.001325 | MSE 0.000002
Processing file: meshes\cylinder.obj

=== cylinder ===
Original: count=192 | min=[-1. -1. -1.] | max=[1. 1. 1.] | mean=[-8.0714626e-09  0.0000000e+00 -1.4280279e-08] | std=[0.70710665 1.         0.7071068 ]
Span was called for V: 
[2. 2. 2.]
Min–Max  | MAE per axis [0.00091642 0.         0.00091642] | MAE 0.000611 | MSE 0.000001
UnitSphere| MAE per axis [0.00138241 0.00138238 0.00138241] | MAE 0.001382 | MSE 0.000003
Processing file: meshes\explosive.obj

=== explosive ===
Original: count=2844 | min=[-0.199625 -0.       -0.197126] | m